# Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/


### Input Data
- One timeseries txt file per participant. Conditions must be separated in unique folder
- Need a txt file of the labels for each ROI

### Outputs
- Circle graph showing thresholded connectivity of each ROI. ROI color denotes modules. 
- Circle graph of module edge weights. 
- CSV file with module assignment & nodal metrics for each ROI

### Running the notebook
The following parts of the code should be changed
1. Filepath to timeseries 
2. Name of circle graph figure
3. Module dict to assign to module graph
4. Name of module graph figure
5. Name of csv file

## Note: 
### Print Statements are commented out to save time, remove comments if desired. 

In [101]:
# Imports 
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import statistics
import pdb
from scipy import stats
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

## Load in the data

### Find the path to the data

In [102]:
#Find the path to data
file_list = glob.glob('/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/*.txt')

In [103]:
#Check the files found
#print(file_list)

In [104]:
#Check to see how many participants 
#len(file_list)

### Make a dictionary to read in the files to pandas

In [105]:
#Setting the ditionary
my_dict={}
for item in file_list:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-001_punish
sub-002_punish
sub-003_punish
sub-004_punish
sub-005_punish
sub-006_punish
sub-007_punish
sub-009_punish
sub-010_punish
sub-011_punish
sub-012_punish
sub-013_punish
sub-014_punish
sub-015_punish
sub-016_punish
sub-017_punish
sub-018_punish
sub-019_punish
sub-020_punish
sub-021_punish
sub-022_punish
sub-024_punish
sub-025_punish
sub-026_punish
sub-027_punish
sub-028_punish
sub-029_punish
sub-030_punish
sub-031_punish
sub-032_punish
sub-033_punish
sub-034_punish
sub-035_punish
sub-036_punish
sub-037_punish
sub-038_punish
sub-039_punish
sub-040_punish
sub-041_punish
sub-042_punish
sub-043_punish
sub-044_punish
sub-045_punish
sub-046_punish
sub-047_punish
sub-048_punish
sub-050_punish
sub-052_punish
sub-053_punish
sub-054_punish
sub-055_punish
sub-056_punish
sub-057_punish
sub-058_punish
sub-059_punish
sub-060_punish
sub-061_punish
sub-062_punish
sub-063_punish
sub-064_punish
sub-066_punish
sub-067_punish
sub-068_punish
sub-069_punish
sub-070_punish
sub-071_punish
sub-072_pu

In [106]:
# Checking to make sure its populated
#my_dict['sub-024_reward']

### Read the data from the dictionary into pandas

In [107]:
#Setting the data dictionary
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

In [108]:
#Check the dictionary
data_dict.keys()

dict_keys(['sub-001_punish', 'sub-002_punish', 'sub-003_punish', 'sub-004_punish', 'sub-005_punish', 'sub-006_punish', 'sub-007_punish', 'sub-009_punish', 'sub-010_punish', 'sub-011_punish', 'sub-012_punish', 'sub-013_punish', 'sub-014_punish', 'sub-015_punish', 'sub-016_punish', 'sub-017_punish', 'sub-018_punish', 'sub-019_punish', 'sub-020_punish', 'sub-021_punish', 'sub-022_punish', 'sub-024_punish', 'sub-025_punish', 'sub-026_punish', 'sub-027_punish', 'sub-028_punish', 'sub-029_punish', 'sub-030_punish', 'sub-031_punish', 'sub-032_punish', 'sub-033_punish', 'sub-034_punish', 'sub-035_punish', 'sub-036_punish', 'sub-037_punish', 'sub-038_punish', 'sub-039_punish', 'sub-040_punish', 'sub-041_punish', 'sub-042_punish', 'sub-043_punish', 'sub-044_punish', 'sub-045_punish', 'sub-046_punish', 'sub-047_punish', 'sub-048_punish', 'sub-050_punish', 'sub-052_punish', 'sub-053_punish', 'sub-054_punish', 'sub-055_punish', 'sub-056_punish', 'sub-057_punish', 'sub-058_punish', 'sub-059_punish',

In [109]:
#Check for the dataframe
#data_dict['sub-058_reward']

### Create a new dictionary with correlation matrix

In [110]:
#Setting up the correlation dictionary
cor_dict={}

for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix
    

In [111]:
#check the dictionary
#list(cor_dict.values())[3]

### Make a dictionary of labels for the nodes

In [112]:
#This points to a txt file with the ROI names in a list separated by commas
path = '/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/labels.txt'
df_label = pd.read_csv(path, sep=',')

#df_label.head()

In [113]:
labels_dict = {}
n=0
for item in df_label:
    labels_dict[n]=item
    n=n+1

In [114]:
#print(labels_dict)

### Function to create a graph with positive or negative values and minimum correlation value

In [115]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)
- The modules (communitites in the network at the participant level

In [116]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    graph_dict={}
    partition_dict={}
    for key, values in list_o_data.items():
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict, 'modules':partition_dict})

### Apply the function to correlations & check output

In [117]:
# Apply function
GRAPHS = make_graphs(cor_dict, "positive", 0)

In [118]:
# Check the keys for the dictionary
#GRAPHS.keys()

In [119]:
# Check modules for one subject
#GRAPHS['modules']['sub-001_reward']

In [120]:
#Check to make sure graphs are filled
#Test = GRAPHS['graphs']['sub-001_reward'][0]
#Test.edges(data=True)

In [121]:
#GRAPHS['mean_FC']

### Get the standard deviation of the mean FC

In [122]:
#statistics.stdev(GRAPHS['mean_FC'])
o=[]

for key,value in GRAPHS['mean_FC'].items():
    o.append(value[0])

In [123]:
statistics.stdev(o)

0.06342245078069726

If this value is low (<0.1) then you don't need to threshold a graph

### Test Modularity

In [124]:
#Goal: use modularity function from communities to identify module structure that 
#emerge during reward within the sample

In [125]:
#this is pulling out the module structure for each participant
modules=[]

for key,value in GRAPHS['modules'].items():
    modules.append(value[0])

In [126]:
#print it to make sure it works
#modules

In [127]:
#made a df of the modules
df = pd.DataFrame.from_dict(modules)

In [128]:
#df

In [129]:
#label the modules
df.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_R", 17:"Oral_somatosensory_cortex_L", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0,1,0,0,2,3,0,0,1,1,...,4,4,6,5,1,1,3,1,6,0
1,0,0,1,1,2,2,0,0,1,1,...,4,1,1,1,4,4,2,2,1,1
2,0,0,1,1,2,2,0,0,3,3,...,2,5,0,0,5,5,4,4,2,1
3,0,0,1,1,2,2,0,0,3,3,...,0,0,1,1,1,1,3,3,1,1
4,0,0,1,1,0,2,0,0,1,3,...,0,1,1,1,4,4,3,3,3,4
5,0,1,2,2,3,1,3,3,0,2,...,5,5,0,0,4,4,0,0,3,3
6,0,0,1,1,2,3,0,0,4,4,...,3,1,0,0,5,5,4,0,2,5
7,0,1,2,2,3,1,1,1,4,4,...,3,3,5,5,6,6,4,4,5,5
8,0,1,2,2,1,0,1,1,3,2,...,5,1,0,0,4,4,3,3,4,0
9,0,0,1,1,0,1,0,0,1,1,...,0,1,2,0,3,0,0,0,0,1


In [130]:
#find the mean over the columns
#df.mean(axis = 0)

In [131]:
# Check the standard deviation over the columns
#df.std(axis = 0)

In [132]:
#find the mean over the columns
partition_median = df.median()
df_median = pd.DataFrame(partition_median)

In [133]:
# View the median module for each node
#df_median

### Make a Dicitionary with the modules

In [134]:
#Convert modules to dictionary
mod_dict={}
for index, row in df_median.iterrows():
    mod_dict[index]= row[0]

#mod_dict

### Combine participant correlation matrices into one mean correlation matrix

In [135]:
#make the function to combine
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.mean(x, axis=(2))
    return(mu)

In [136]:
#Make the mean graph with correlations
mean_graph = make_total_graphs(GRAPHS['correlations'])

In [137]:
#Check to make sure this worked 
mean_graph.shape

#Convert the graph to a numpy matrix so it can be recognized by networkX
mean_graph_mat = np.matrix(mean_graph)

#Check the mean correlation to use to threshold later
mean_graph_mat.mean()

0.3047133910764402

In [138]:
meanG = nx.from_numpy_matrix(mean_graph_mat)

In [139]:
#Add the modules and ROI labels to the graph
nx.set_node_attributes(meanG, mod_dict, 'modules')
nx.set_node_attributes(meanG, labels_dict, 'ROIs')

In [140]:
# Check to make sure this worked
ROIs=nx.get_node_attributes(meanG,'ROIs')
ROIs

{0: 'Amygdala_L',
 1: 'Amygdala_R',
 2: 'Dorsal_striatum_L',
 3: 'Dorsal_striatum_R',
 4: 'Fusiform_gyrus_L',
 5: 'Fusiform_gyrus_R',
 6: 'Hippocampus_L',
 7: 'Hippocampus_R',
 8: 'Insula_L',
 9: 'Insula_R',
 10: 'Intracalcarine_cortex_L',
 11: 'Intracalcarine_cortex_R',
 12: 'lOFC_L',
 13: 'lOFC_R',
 14: 'mOFC_L',
 15: 'mOFC_R',
 16: 'Oral_somatosensory_cortex_R',
 17: 'Oral_somatosensory_cortex_L',
 18: 'Precuneus_L',
 19: 'Precuneus_R',
 20: 'Ventral_striatum_L',
 21: 'Ventral_striatum_R',
 22: 'vlPFC_L',
 23: 'vlPFC_R',
 24: 'vlThalamus_L',
 25: 'vlThalamus_R',
 26: 'vmPFC_L',
 27: 'vmPFC_R'}

### Function to make a thresholded graph

In [141]:
def threshold(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

In [142]:
threshG = threshold(meanG, 'positive', .3)

### Function to visualize thresholded graph with modules in colors

In [143]:
def jenny_graph(graph):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items()) #if your modules are named different change here
    nodes, positions = zip(*nx.get_node_attributes(graph,'ROIs').items())
    #positions
    positions=nx.circular_layout(graph) #this is defining a circluar graph, if you want a different one you change the circular part of this line
    
    #Figure size
    plt.figure(figsize=(40,25))
    
    
    #draws nodes
    color = np.array(color)
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=color,
                           node_size=4000,
                           alpha=0.8, 
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, labels = ROIs, font_size=25, 
                            font_family='sans-serif', fontweight = 'bold')
    
    
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.bwr #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    wt=weights*5
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Modularity',fontsize = 25)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)

    plt.axis('off')
    plt.title("Modularity and Edge Weights of Average Graph", fontsize = 30)
    plt.savefig("/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/modularity_circle_reward.png", format="PNG")
    #plt.show()

In [144]:
jenny_graph(threshG)

### Make Module Graph with Nodes Weighted by Clustering Coefficient

Make a new graph with modules from whole group analysis 

In [145]:
modg = community.induced_graph(mod_dict, meanG)

In [146]:
#Clustering coeff multiplied by 2000 -- so the size works with networkx node sizing
reward_clust_dict = {
    0.0: 1200,
    1.0: 1320,
    2.0: 1360,
    3.0: 1160
}

punish_clust_dict = {
    0.0: 1200,
    1.0: 1280,
    2.0: 1240,
    3.0: 1300
}

In [147]:
#Set module attributes**** Need to change this depending on the condition

#REWARD
nx.set_node_attributes(modg, reward_clust_dict, 'clustering')

#PUNISHMENT
#nx.set_node_attributes(modg, punish_clust_dict, 'clustering')

In [148]:
#list(modg.nodes)

### Function to make module structure graph

In [149]:
def module_fig(G):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())
    
    
    positions=nx.circular_layout(G)
    plt.figure(figsize=(25,20))
    
    color = np.array(list(G.nodes))
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    nx.draw_networkx_nodes(G,positions, 
                           node_color=color, 
                           node_size=size,
                           alpha=1.0, 
                           cmap= 'Spectral',
                           vmin=cm,vmax=cM )
   
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, font_family='sans-serif')
    
    x=nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   
    edge_bar=plt.colorbar(x)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)
    
    plt.title("Module Connectivity Weights", fontsize = 30)
    plt.savefig("/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/results/modularity_edges_reward_weighted.png", format="PNG")
    plt.axis('off')
    #plt.show()

In [150]:
module_fig(modg)

### Calculate Nodal Metrics by Participant
- Clustering Coefficient
- Betweenness centrality

Function to pull the nodal parameters by subject

In [ ]:
def make_node_stats(list_o_data, direction, min_cor):
    clustering_dict ={}
    centrality_dict ={}
    
    
    for key, values in list_o_data.items():
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            tG = create_corr_network_5(G, direction,min_cor)
            ########################################
            clustering = nx.clustering(tG)
            ########################################
            clustering_dict.setdefault(key, []).append(clustering)
            ########################################
            centrality = nx.betweenness_centrality(tG)
            ########################################
            centrality_dict.setdefault(key, []).append(centrality)
            ########################################
    return({'clustering_coeff':clustering_dict, 'btn_centrality':centrality_dict})

In [ ]:
#Metrics computed on an UNTHRESHOLDED GRAPH
nodes = make_node_stats(cor_dict, "positive", 0)

In [ ]:
#nodes.keys()

In [ ]:
#nodes['btn_centrality']['sub-001_reward']

In [ ]:
clustering_coeff=[]

for key,value in nodes['clustering_coeff'].items():
    clustering_coeff.append(value[0])

In [ ]:
df_clust = pd.DataFrame.from_dict(clustering_coeff)
df_clust.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

In [ ]:
central=[]

for key,value in nodes['btn_centrality'].items():
    central.append(value[0])

In [ ]:
df_cent = pd.DataFrame.from_dict(central)
df_cent.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

Save out to CSV files

In [ ]:
df_clust.to_csv('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/node_clustering_reward.csv', header = True, index = None)

In [ ]:
df_cent.to_csv('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/node_centrality_reward.csv', header = True, index = None)

## Save GRAPHS dictionary in a pickle file in case of crash

In [ ]:
pickle.dump(GRAPHS, open('/Users/jennygilbert/Documents/betaseries_bevel/tmp/Graphs', 'wb'), protocol=4)

In [ ]:
with open('/Users/jennygilbert/Documents/betaseries_bevel/tmp/Graphs', 'rb') as pickle_file:
    try:
        while True:
            GRAPHS = pickle.load(pickle_file)
#             print (GRAPHS)
    except EOFError:
        pass